## Knapsack Problem (Zero-One Version)

__Inputs:__ Weight limit $W$, list of item weights $[w_1, \ldots, w_k]$, and list of item values $[v_1, \ldots, v_k]$.

__Output:__ For each item, we can choose it in our Knapack $n_i = 1$ or leave it out of our knapsack $n_i = 0$ so that  
   1. Total weight is under the knapsack weight limit: $n_1 w_1 + \cdots + n_k w_k \leq W$. Note here that each $n_i \in \{ 0, 1\} $, depending on whether the item \# i is chosen or not.
   2. The value of stolen goods is maximized: $n_1 v_1 + \ldots + n_k v_k $ is max.

In [ ]:
# Important, Run this cell below
W = 200 # weight limit is 200
weights = [1, 5, 20, 35, 90] # These are the weights of individual items
values = [15, 14.5, 19.2, 19.8, 195.2] # These are the values of individual items

## 1. Identify the optimal substructure

Suppose the current weight limit is $W$ and we have made decisions for all items from $1, \ldots, j-1$, where $j \geq 1$.  What decisions can we make for Item \# $j$? 

   1. Steal item $j$: remaining weight limit is $W - w_j$ and we have gained a value of $v_j$. The __remaining problem__ is to find the best way to steal for weight limit $W - w_j$ with items from $j+1, \ldots, n$.
   2. Do __not__ steal item $j$: remaining weight limit is still $W $ and we have gained no value since we did not steal item $j$. The __remaining problem__ is to find the best way to steal for weight limit $W$ with items from $j+1, \ldots, n$.
   
We can thus see that the problem has optimal substructure:
 - We can make the decisions in _stages_, in this case one item at a time.
 - Once we make a decision, the remaining problem is also an instance of the original problem but the data changes.

## 2. Recurrence
$$\newcommand\msz{\text{maxStealZeroOne}}$$
$$\msz(W, j) = \max\ \left\{ \begin{array}{ll}
v_j + \msz(W - w_j, j+1), & \leftarrow \ \text{steal # j} \\ 
\msz(W, j+1) & \leftarrow\ \text{skip # j} \\ 
\end{array} \right.$$

Base Cases:

  * $\msz(0, j) = 0$, for all $j \in \{1,\ldots, n\}$. This handles the case when we have 0 weight capacity left.
  * $\msz(W, j) = -\infty$ if $W < 0$, for all $j \in \{1,\ldots, n\}$. This handles the case when we have violated our weight capacity constraints.
  * $\msz(W, n+1) = 0$ for all $W \geq 0$.  This handles the case when we have run out of items to steal. 
 
 


In [ ]:
def maxStealZeroOne(W, j, weights, values):
    assert j >= 0 
    assert len(weights) == len(values)
    # weights -- list of item weights
    # values -- list of item values
    # W weight limit
    # j item number we are considering.

    # First the base cases
    if W == 0: 
        return 0
    if W < 0: # we have added more items to knapsack than its original capacity
        return -float('inf')
    if j >= len(weights): 
        return 0
    # Next, handle the recurrence.
    return max(
        values[j] + maxStealZeroOne(W - weights[j], j+1, weights, values),  # steal item j
        maxStealZeroOne(W, j+1, weights, values)# skip item j
               )

In [ ]:
maxStealZeroOne(W, 0, weights, values)

263.7

In [ ]:
maxStealZeroOne(20, 0, weights, values)

29.5

## 3. Memoize

Memoization of the recurrence $\msz$ will convert it to a table. 
 - Table entry $T[(w, j)]$ will represent the value of $\msz(w,j)$ for weight limit $0 \leq w \leq W$ and $1 \leq j \leq n$. 
 - We will assume that $T[(0, *)] = 0$ and $T[(*, n+1)] = 0$, where * just denotes an arbitrary number for that argument. 
 - If we tried to access $T[(w, *)]$ for negative $w < 0$, we will assume it evaluates to $-\infty$. 


## 4. Recover Solution

We store in a separate table $S[(0,0)], \ldots, S[(W,n)]$ which option provides us with the best value: 
  - $S[(w, j)] = +1$ means that for weight limit $w$, we will choose to include item $j$.
  - $S[(w,j)] = 0$ means that for weight limit $w$, we will skip item $j$. 

The goal will be to first fill out the tables $T, S$ for given problem inputs and then recover solution.

Recall the recurrence once again: 
$$\msz(W, j) = \max\ \left\{ \begin{array}{ll}
v_j + \msz(W - w_j, j+1), & \leftarrow \ \text{steal # j} \\ 
\msz(W, j+1) & \leftarrow\ \text{skip # j} \\ 
\end{array} \right.$$

We see that $\msz(w,j)$ requires us to know $\msz(w', j+1)$ for $w' \leq w$. 
 - Therefore, the table must be filled from $w = 0, \ldots, W$ in ascending order and $j = n, \ldots, 1$ in descending order. 
 
 This is important to note for our memoization algorithm. 


In [ ]:
def memoizedMaxStealZeroOne(W, weights, values): 
    n = len(weights)
    assert (len(values) == n), 'Weights and Values list must be of same size'
    assert (W >= 0)
    if W == 0: 
        return 0, []# nothing to steal and 0 value derived.
    
    # Initialize the memo table as a list of lists
    # fill in all entries with a zero
    T = [ [0 for j in range(n)] for w in range(W+1)]
    S = [ [0 for j in range(n)] for w in range(W+1)]

    # we will use this helper method to access our memo table.
    # it will save us a lot of code later.
    def getTblEntry(w, j): 
        if w == 0: 
            return 0
        if w < 0: 
            return -float('inf')
        if j >= n:
            return 0
        return T[w][j]

    for w in range(1, W+1): # w in ascending order from 1 to W.
        for j in range(n-1, -1, -1):  # this is a descending order loop from n-1 to 0.
            # this allows us to simultaneously fill T, S without using if-then-else loop
            (T[w][j], S[w][j]) = max(
                (values[j] + getTblEntry(w - weights[j], j+1), 1), 
                (getTblEntry(w, j+1), 0))
    itemsToSteal = [] 
    # recover solution
    weightOfKnapsack = W  
    for j in range(n): 
        if (S[weightOfKnapsack][j] == 1):
            itemsToSteal.append(j)
            weightOfKnapsack = weightOfKnapsack - weights[j]
            print(f'Steal Item {j}: Weight = {weights[j]}, Value = {values[j]}')
    print(f'Total weight stolen: {W - weightOfKnapsack}, value = {T[W][0]}')
    return (T[W][0], itemsToSteal)
            
    
        

In [ ]:
memoizedMaxStealZeroOne(W, weights, values)

Steal Item 0: Weight = 1, Value = 15
Steal Item 1: Weight = 5, Value = 14.5
Steal Item 2: Weight = 20, Value = 19.2
Steal Item 3: Weight = 35, Value = 19.8
Steal Item 4: Weight = 90, Value = 195.2
Total weight stolen: 151, value = 263.7


(263.7, [0, 1, 2, 3, 4])

In [ ]:
memoizedMaxStealZeroOne(20, weights, values)

Steal Item 0: Weight = 1, Value = 15
Steal Item 1: Weight = 5, Value = 14.5
Total weight stolen: 6, value = 29.5


(29.5, [0, 1])

In [ ]:
memoizedMaxStealZeroOne(150, weights, values)

Steal Item 0: Weight = 1, Value = 15
Steal Item 2: Weight = 20, Value = 19.2
Steal Item 3: Weight = 35, Value = 19.8
Steal Item 4: Weight = 90, Value = 195.2
Total weight stolen: 146, value = 249.2


(249.2, [0, 2, 3, 4])

# Knapsack Problem with Unbounded Number of Items

We will study a version of Knapsack where the user can choose each item an unbounded number of times.

__Inputs:__ Weight limit $W$, list of item weights $[w_1, \ldots, w_k]$, and list of item values $[v_1, \ldots, v_k]$.

__Output:__ Choose how many of each item to take $[n_1, \ldots, n_k]$ so that 
   1. Total weight is under the knapsack weight limit: $n_1 w_1 + \cdots + n_k w_k \leq W$.
   2. The value of stolen goods is maximized: $n_1 v_1 + \ldots + n_k v_k $ is max.

In [ ]:
W = 200
weights = [1, 5, 20, 35, 90]
values = [15, 14.5, 19.2, 19.8, 195.2]

## 1. Identify the optimal substructure

Suppose the current weight limit is $W$. Let us commit to stealing one of the available items and look at what is left to do.

   1. Suppose we commit to stealing item $j$.
   2. We now need to solve the same problem but for weight limit $W - w_j$. If the solution for this subproblem is obtained, then the original problem's solution is to take the solution for $W-w_j$ and append item $j$ to it.
   
We can thus see that the problem has optimal substructure.

## 2. Recurrence

$$\text{maxSteal}(W) = \max\ \left\{ \begin{array}{ll}
0 & \leftarrow\ \text{Choose to steal nothing and Quit!}\\
v_1 + \text{maxSteal}(W - w_1) & \leftarrow\ \text{Choose one unit of item}\ 1 \\
v_2 + \text{maxSteal}(W - w_2) & \leftarrow\ \text{Choose one unit of item}\ 2 \\
\vdots & \\
v_k + \text{maxSteal}(W - w_k) & \leftarrow\ \text{Choose one unit of item}\ k\\
\end{array} \right.$$

Base Case:

  * $\text{maxSteal}(0) = 0$ 
  * $\text{maxSteal}(W) = -\infty$ if $W < 0$.
 
 


In [ ]:
def maxSteal(W, weights, values):
    if W == 0: 
        return 0
    if W < 0:
        return -float('inf')
    k = len(weights)
    assert len(values) == k
    opts = [ values[i] + maxSteal(W - weights[i], weights, values) for i in range(k) ]
    return max(opts)

In [ ]:
print(maxSteal(25, weights, values))
#WARNING: This will run for a very very long time.
#print(maxSteal(W, weights, values))

375


## 3. Memoize

Memoization is very simple. We make a table $T[0], ... , T[W]$ for storing $\text{maxSteal}(j)$ for j ranging from $0$ to $W$.
The rest just follows the structure of the recurrence taking care to handle -ve values for weight separately.

## 4. Recover Solution

We store in a separate table $S[0], \ldots, S[W]$ which option provides us with the best value.


In [ ]:
def maxSteal_memo(W, weights, values):
    # Initialize the tables
    T = [0]* (W+1)
    S = [-1]* (W+1)
    k = len(weights)
    assert len(values) == k
    for w in range(1, W+1):
        opts =  [  ( (values[i]+ T[ w - weights[i] ]), i )  for i in range(k) if w - weights[i] >= 0 ]
        opts.append( (-float('inf'), -1) ) # In case opts was empty from the previous step.
        T[w], S[w] = max(opts)
    # This finishes the computation
    stolen_item_ids = []
    weight_remaining = W
    while weight_remaining >= 0:
        item_id = S[weight_remaining]
        if item_id >= 0:
            stolen_item_ids.append('Steal Item ID %d: weight = %d, value = %f' % (item_id, weights[item_id], values[item_id]) )
            weight_remaining = weight_remaining - weights[item_id]
        else:
            break
    return T[W], stolen_item_ids

In [ ]:
print(maxSteal_memo(25, weights, values))
print(maxSteal_memo(W, weights, values))

(375, ['Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.000000', 'Steal Item ID 0: weight = 1, value = 15.00